In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
data=pd.read_csv("day.csv")

### Get a Feel of Data

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.head()

### Remove Unnecessary Columns
 - instant - As this column contains all unique values
 - dteday - As this data is already present in separate columns i.e dteday
 - casual & registered - As this sum is equal to cnt variable i.e Target we are predicting and would result in High correlation

In [ ]:
data.drop(['instant','dteday','casual','registered'], axis=1,inplace=True)

In [ ]:
data.head()

### Data Visualization on Categorical Variables

Analyze the Categorical Variable on the target Variable i.e cnt

 - season
 - yr
 - mnth
 - workingday
 - weathersit

From the below graphs, all the categorical variables have good amount of impact on the cnt

In [ ]:
plt.figure(figsize=(20,12))
plt.subplot(2,3,1)
sns.boxplot(x='season',y='cnt',data=data)
plt.subplot(2,3,2)
sns.boxplot(x='yr',y='cnt',data=data)
plt.subplot(2,3,3)
sns.boxplot(x='mnth',y='cnt',data=data)
plt.subplot(2,3,4)
sns.boxplot(x='workingday',y='cnt',data=data)
plt.subplot(2,3,5)
sns.boxplot(x='weathersit',y='cnt',data=data)
plt.show()

### Data Visualization on Continous Variables

 - temp
 - atemp
 - hum
 - windspeed
 - cnt 
From the below graph, cnt vs temp and cnt vs temp looks to be linear relationship which is not the case with cnt vs hum 
and windspeed

Also from the graph looks like temp and atemp looks to related i.e chances of multi-collinearity


In [ ]:
columns=['temp','atemp','hum','windspeed','cnt']
sns.pairplot(data[columns])
plt.show()

### Data Preparation

### One Hot Encoding for Categorical Independent Variables

Apply one hot encoding technique (i.e Create Dummy Variables) to the following categorical variables 

 - season. Convert numerical values to relavent values and apply one hot encoding
 - month
 - weekday
 - weathersit. Convert numerical values to relavent values and apply one hot encoding

### Create Dummy Variable for Season
 - Map value in season to actual season name as given in data dictionary 1:spring, 2:summer, 3:fall, 4:winter
 - Create dummy variable for Season
 - Add the dummy variables to the original data frame
 - Drop the original season column

In [ ]:
data.season.value_counts()

In [ ]:
data['season']=data[['season']].apply(lambda x : x.map({1:"spring", 2:"summer", 3:"fall", 4:"winter"}))
season_dummy=pd.get_dummies(data['season'],drop_first=True,dtype=int)
data=pd.concat([data,season_dummy],axis=1)


In [ ]:
data.season.value_counts()

In [ ]:
data=data.drop("season",axis=1)
data.head()

### Create Dummy Variable for Month

 - Map value in Mnth to actual month name 
 - Create dummy variable for Mnth
 - Add the dummy variables to the original data frame
 - Drop the original mnth column

In [ ]:
data.mnth.value_counts()

In [ ]:
data['mnth']=data[['mnth']].apply(lambda x : x.map({1:"Jan", 2:"Feb", 3:"Mar", 4:"Apr",5:"May", 6:"Jun",
                                                   7: "Jul",8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov",
                                                   12: "Dec"
                                                   })
                                 )
mnth_dummy=pd.get_dummies(data['mnth'],drop_first=True,dtype=int)
data=pd.concat([data,mnth_dummy],axis=1)


In [ ]:
data.head()

In [ ]:
data.mnth.value_counts()

In [ ]:
data=data.drop("mnth",axis=1)
data.head()

### Create Dummy Variable for Weekday

 - Map value in weekday to actual day
 - Create dummy variable for weekday
 - Add the dummy variables to the original data frame
 - Drop the original weekday column

In [ ]:
data.weekday.value_counts()

In [ ]:
data['weekday']=data[['weekday']].apply(lambda x : x.map({0:"Sunday", 1:"Monday", 2:"Tuesday", 3:"Wednesday", 4:"Thursday",
                                                          5:"Friday", 6:"Saturday"
                                                           })
                                         )
weekday_dummy=pd.get_dummies(data['weekday'],drop_first=True,dtype=int)
data=pd.concat([data,weekday_dummy],axis=1)


In [ ]:
data.head()

In [ ]:
data.weekday.value_counts()

In [ ]:
data=data.drop("weekday",axis=1)
data.head()

### Create Dummy Variable for WeatherSit

 - Map value in weathersit to actual weathersit as given in data dictionary in a meaningful manner
 - Create dummy variable for weathersit
 - Add the dummy variables to the original data frame
 - Drop the original weathersit column

In [ ]:
data.weathersit.value_counts()

In [ ]:
data['weathersit']=data[['weathersit']].apply(lambda x : x.map({1:"Clear", 2:"Mist", 3:"Light", 4:"Heavy"}))
weathersit_dummy=pd.get_dummies(data['weathersit'],drop_first=True,dtype=int)
data=pd.concat([data,weathersit_dummy],axis=1)


In [ ]:
data.weathersit.value_counts()

In [ ]:
data.head()

In [ ]:
data=data.drop("weathersit",axis=1)
data.head()

### Check the list of columns

In [ ]:
data.dtypes

### Split the data to Train and Test with 70-30 ratio

In [ ]:
data_train, data_test=train_test_split(data, train_size=0.7, random_state=99)

In [ ]:
data_train.shape

In [ ]:
data_test.shape

### Applying Min Max Scaling on numeric variables so that they all are at same scale
 - temp
 - atemp
 - hum
 - windspeed
 - cnt

In [ ]:
scaler = MinMaxScaler()

In [ ]:
numeric_vars=['temp','atemp','hum','windspeed','cnt']

In [ ]:
data_train[numeric_vars]=scaler.fit_transform(data_train[numeric_vars])

In [ ]:
data_train.head()

In [ ]:
data_train.describe()

### Training the model

In [ ]:
plt.figure(figsize=(24,14))
sns.heatmap(data_train.corr(),annot=True)
plt.show()

In [ ]:
y_train=data_train.pop('cnt')
x_train=data_train
x_train_sm=sm.add_constant(x_train)
lr=sm.OLS(y_train,x_train_sm)
lr_model=lr.fit()
lr_model.summary()

### Identify the variables to keep in the model based on below stats
 - p-value
 - VIF


In [ ]:
vif=pd.DataFrame()
vif['Features']=x_train.columns
vif['VIF']= [ variance_inflation_factor(x_train.values,i) for i in range(x_train.shape[1]) ]
vif['VIF']=round(vif['VIF'],2)
vif=vif.sort_values(by='VIF',ascending=False)
vif

### Remove variables with High p(i.e > 0.05 CI) and VIF(i.e >5) values Ex: atemp

In [ ]:
x_train=data_train.drop(['atemp'],axis=1)
x_train_sm=sm.add_constant(x_train)
lr=sm.OLS(y_train,x_train_sm)
lr_model=lr.fit()
lr_model.summary()

In [ ]:
vif=pd.DataFrame()
vif['Features']=x_train.columns
vif['VIF']= [ variance_inflation_factor(x_train.values,i) for i in range(x_train.shape[1]) ]
vif['VIF']=round(vif['VIF'],2)
vif=vif.sort_values(by='VIF',ascending=False)
vif

### Remove variables with High p and Low VIF and rebuild the model and observe if there is any improvement in model . Ex: Wednesday, Thursday

In [ ]:
x_train=data_train.drop(['Wednesday','atemp'],axis=1)
x_train_sm=sm.add_constant(x_train)
lr=sm.OLS(y_train,x_train_sm)
lr_model=lr.fit()
lr_model.summary()

In [ ]:
vif=pd.DataFrame()
vif['Features']=x_train.columns
vif['VIF']= [ variance_inflation_factor(x_train.values,i) for i in range(x_train.shape[1]) ]
vif['VIF']=round(vif['VIF'],2)
vif=vif.sort_values(by='VIF',ascending=False)
vif

In [ ]:
x_train=data_train.drop(['Wednesday','atemp','Thursday'],axis=1)
x_train_sm=sm.add_constant(x_train)
lr=sm.OLS(y_train,x_train_sm)
lr_model=lr.fit()
lr_model.summary()

In [ ]:
vif=pd.DataFrame()
vif['Features']=x_train.columns
vif['VIF']= [ variance_inflation_factor(x_train.values,i) for i in range(x_train.shape[1]) ]
vif['VIF']=round(vif['VIF'],2)
vif=vif.sort_values(by='VIF',ascending=False)
vif

### Selection of Variables using Recursive Feature Selection(RFE)
Instead of performing these iteration by removing variables one by one, we will use RFE

In [ ]:
x_train=data_train
lm = LinearRegression()
lm.fit(x_train,y_train)

rfe = RFE(lm,n_features_to_select=15)
rfe = rfe.fit(x_train,y_train)

In [ ]:
list(zip(x_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
columns=x_train.columns[rfe.support_]
columns

In [ ]:
x_train_rfe=data_train[columns]
x_train_sm=sm.add_constant(x_train_rfe)
lr=sm.OLS(y_train,x_train_sm)
lr_model=lr.fit()
lr_model.summary()

In [ ]:
vif=pd.DataFrame()
vif['Features']=x_train_rfe.columns
vif['VIF']= [ variance_inflation_factor(x_train_rfe.values,i) for i in range(x_train_rfe.shape[1]) ]
vif['VIF']=round(vif['VIF'],2)
vif=vif.sort_values(by='VIF',ascending=False)
vif

In [ ]:
## Remove variable atemp as it has high p-value and high vif

In [ ]:
x_train_rfe=data_train[columns].drop(['atemp'],axis=1)
x_train_sm=sm.add_constant(x_train_rfe)
lr=sm.OLS(y_train,x_train_sm)
lr_model=lr.fit()
lr_model.summary()

In [ ]:
vif=pd.DataFrame()
vif['Features']=x_train_rfe.columns
vif['VIF']= [ variance_inflation_factor(x_train_rfe.values,i) for i in range(x_train_rfe.shape[1]) ]
vif['VIF']=round(vif['VIF'],2)
vif=vif.sort_values(by='VIF',ascending=False)
vif

## Remove variable hum,Jul as it has high vif

In [ ]:
x_train_rfe=data_train[columns].drop(['atemp','hum','Jul'],axis=1)
x_train_sm=sm.add_constant(x_train_rfe)
lr=sm.OLS(y_train,x_train_sm)
lr_model=lr.fit()
lr_model.summary()

In [ ]:
vif=pd.DataFrame()
vif['Features']=x_train_rfe.columns
vif['VIF']= [ variance_inflation_factor(x_train_rfe.values,i) for i in range(x_train_rfe.shape[1]) ]
vif['VIF']=round(vif['VIF'],2)
vif=vif.sort_values(by='VIF',ascending=False)
vif

In [ ]:
data_train.head()

### Residual Analyis on Train Data
The graph shows that it is centered around zero mean and is normally distributed

In [ ]:
y_train_cnt=lr_model.predict(x_train_sm)

In [ ]:
fig=plt.figure()
sns.displot(y_train-y_train_cnt,bins=20)
fig.suptitle('Error Terms',fontsize=20)
plt.xlabel('Errors',fontsize=18)


### Prediction

In [ ]:
numeric_vars=['temp','atemp','hum','windspeed','cnt']
df_test=data_test
df_test[numeric_vars]=scaler.transform(df_test[numeric_vars])

In [ ]:
y_test=df_test.pop('cnt')
x_test=df_test[columns].drop(['atemp','hum','Jul'],axis=1)

In [ ]:
df_test

In [ ]:
x_test_sm=sm.add_constant(x_test)
y_test_cnt=lr_model.predict(x_test_sm)

### Model Evaluation

R-square on the test dataset is very much close  to the r-square on the trained dataset i.e 85%

In [ ]:
r2_score(y_true=y_test,y_pred=y_test_cnt)